In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os


In [ ]:
# Useful functions
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

def save_model(model):
    json_string = model.to_json()
    if not os.path.isdir('cache'):
        os.mkdir('cache')
    open(os.path.join('cache', 'architecture.json'), 'w').write(json_string)
    model.save_weights(os.path.join('cache', 'model_weights.h5'), overwrite=True)


def read_model():
    model = model_from_json(open(os.path.join('cache', 'architecture.json')).read())
    model.load_weights(os.path.join('cache', 'model_weights.h5'))
    return model

def create_submission(predictions, test_id):
    result = pd.DataFrame(predictions, columns=['id', 'label'])
    result.loc[:, 'img'] = pd.Series(test_id, index=result.index)
    if not os.path.isdir('subm'):
        os.mkdir('subm')

def is_cleaned(pred_value):
    if pred_value < 0.5:
        return 'cleaned'
    else:
        return 'dirty'


In [ ]:
train_dir = 'superbowllsh/train'
test_dir = 'superbowllsh/test'
img_width, img_height = 341, 256

input_shape = (img_width, img_height, 3)
epochs = 30
batch_size = 16

numb_train_samples = 40
numb_test_samples = 660

In [1]:
# Архитектура сети
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [ ]:
# Uploading images
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# Training
model.fit_generator(
    train_generator,
    steps_per_epoch=numb_train_samples // batch_size,
    epochs=epochs)
save_model(model)

In [ ]:
# Get result
read_model()
    
result = pd.DataFrame({'id': [], 'label': []})
i = 0
for filename in os.listdir(test_dir):
    if filename.endswith(".jpg"): 
        img_id, _ = filename.split('.')
        img_path = 'superbowllsh/test/' + filename
        new_image = load_image(img_path)
        pred = model.predict(new_image)
        
        result.loc[i] = [img_id] + [is_cleaned(pred)]
        i += 1

print(result)
        
        

In [ ]:
# Save result to csv file
result = result.sort_values(by=['id'])
result.to_csv('submission.csv', index=False)